# PV System Distribution

In [1]:
# pyathena
from pyathena.connection import Connection
from pyathena.pandas_cursor import PandasCursor

In [2]:
s3_staging_dir = "s3://nrel-tests/tracking_the_sun/"
table_name = "oedi.oedi_dev_tracking_the_sun"

In [3]:
cursor = Connection(region_name="us-west-2", s3_staging_dir=s3_staging_dir).cursor(PandasCursor)

## State Level

In [4]:
import math

# folium
from branca import colormap
from branca.element import Template, MacroElement
import folium
from folium import plugins

# pandas & geopandas
import pandas
import geopandas

# shapely
from shapely import geometry

In [5]:
pv_state = cursor.execute(
    f"""
    SELECT state, COUNT(*) AS count
    FROM {table_name}
    GROUP by state;
    """
).as_pandas()

In [6]:
# convert pandas dataframe to geodataframe
geo_state = geopandas.read_file("us-states.geojson")[["id", "geometry"]]
geo_state.rename(columns={"id": "state"}, inplace=True)

geo_pv_state = geopandas.GeoDataFrame(
    data=pv_state.merge(geo_state, on="state"),
    geometry="geometry",
    crs={"init": "epsg:4326"}
)

In [7]:
# Display state pv system numbers on map
state_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="OpenStreetMap")

tooltip = folium.GeoJsonTooltip(
    fields=["count"],
    aliases=["count:"],
    labels=True,
    sticky=False
)

colors = ["#ffffcc","#ffeda0","#fed976","#feb24c","#fd8d3c","#fc4e2a","#e31a1c","#bd0026","#800026"]
bins = [0, 100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000]
colorscale = colormap.StepColormap(
    colors=colors,
    index=bins,
    vmin=0,
    vmax=1000000
)
def style_function(feature):
    count = feature["properties"]["count"]
    return {
        "color": "#000000",
        "weight": 0.2,
        "opacity": 0.6,
        "fillColor": colorscale(count),
        "fillOpacity": 0.4,
    }

folium.GeoJson(
    name="Distribution of State PV Systems",
    data=geo_pv_state.to_json(),
    tooltip=tooltip,
    style_function=style_function
).add_to(state_map)

legend = MacroElement()
with open("1_pv_distribution_legend.html") as f:
    template = f.read()
legend._template = Template(template)
state_map.get_root().add_child(legend)

state_map

## County Level

In [8]:
pv_county = cursor.execute(
    f"""
    SELECT state, county, COUNT(*) AS count
    FROM {table_name}
    GROUP by state, county;
    """
).as_pandas()

In [9]:
pv_county.head()

,state,county,count
0,PA,Greene,5
1,PA,Lackawanna,66
2,WI,Oconto,24
3,WI,Green,26
4,ME,-9999,555


In [10]:
# convert pandas dataframe to geodataframe
geo_county = geopandas.read_file("us-counties.geojson")[["state", "county", "geometry"]]

# WARNING: pv_county contains invalid county name '-9999', use inner join here.
geo_pv_county = geopandas.GeoDataFrame(
    data=pandas.merge(pv_county, geo_county, how="inner", left_on=["state", "county"], right_on=["state", "county"]),
    geometry="geometry",
    crs={"init": "epsg:4326"}
)

In [11]:
# Display county pv system numbers on map
county_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="OpenStreetMap")

tooltip = folium.GeoJsonTooltip(
    fields=["count"],
    aliases=["count:"],
    labels=True,
    sticky=False
)

colorscale = colormap.LinearColormap(
    colors=["#ffffcc","#ffeda0","#fed976","#feb24c","#fd8d3c","#fc4e2a","#e31a1c","#bd0026","#800026"],
    index=[100, 500, 1000, 5000, 10000, 50000, 100000, 500000, 1000000]
)
def style_function(feature):
    count = feature["properties"]["count"]
    return {
        "color": "#000000",
        "weight": 0.2,
        "opacity": 0.6,
        "fillColor": colorscale(count),
        "fillOpacity": 0.4,
    }

folium.GeoJson(
    name="Distribution of State PV Systems",
    data=geo_pv_county.to_json(),
    tooltip=tooltip,
    style_function=style_function
).add_to(county_map)

legend = MacroElement()
with open("1_pv_distribution_legend.html") as f:
    template = f.read()
legend._template = Template(template)
county_map.get_root().add_child(legend)

county_map

## City Level

In [12]:
pv_city = cursor.execute(
    f"""
    SELECT state, county, city, COUNT(*) AS count
    FROM {table_name}
    GROUP by state, county, city;
    """
).as_pandas()

In [14]:
geo_city = geopandas.read_file("us-cities.geojson")[["state", "county", "city", "geometry"]]

In [15]:
# Handle case
pv_city["city"] = pv_city["city"].str.title()
pv_city["county"] = pv_city["county"].str.title()

geo_city["city"] = geo_city["city"].str.title()
geo_city["county"] = geo_city["county"].str.title()

# Merge
geo_pv_city = geopandas.GeoDataFrame(
    data=pandas.merge(pv_city, geo_city, how="inner", left_on=["state", "county", "city"], right_on=["state", "county", "city"]),
    geometry="geometry",
    crs={"init": "epsg:4326"}
)

In [16]:
# Display city pv system numbers on map
city_map = folium.Map(location=[39.8283, -98.5795], zoom_start=4, tiles="cartodbpositron")

# Marker Cluster
coordinates = [[geom.y, geom.x] for geom in geo_pv_city.geometry.values]
marker_cluster = plugins.FastMarkerCluster(data=coordinates)
marker_cluster.add_to(city_map)

# Heatmap
heat_map = plugins.HeatMap(coordinates, radius=15)
heat_map.add_to(city_map)

city_map